In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, rand
import time

# Crear un DataFrame grande de prueba con 20 millones de filas
df = spark.range(0, 20_000_000).withColumn("valor", (rand() * 1000).cast("int"))
# Ver cuántas particiones tiene inicialmente
#print(f"Particiones iniciales: {df.rdd.getNumPartitions()}")
# Reparticionar el DataFrame a un número mayor de particiones
df_repartitioned = df.repartition(20)
#print(f"Después de repartition(20): {df_repartitioned.rdd.getNumPartitions()}")
# Escribir en disco y medir el tiempo
start = time.time()
df_repartitioned.write.mode("overwrite").parquet("dbfs:/content/sample_data/dataset_repartitioned")
end = time.time()


In [0]:
print(f"Tiempo de escritura con repartition: {end - start:.2f} segundos")
# Leer el dataset con muchas particiones
df_read = spark.read.parquet("/content/sample_data/dataset_repartitioned")
print(f"Particiones al leer: {df_read.rdd.getNumPartitions()}")

# Reducir el número de particiones con coalesce
df_coalesced = df_read.coalesce(1)
print(f"Después de coalesce(1): {df_coalesced.rdd.getNumPartitions()}")
# Escribir en disco y medir el tiempo
start = time.time()
df_coalesced.write.mode("overwrite").parquet("/tmp/dataset_coalesced")
end = time.time()
print(f"Tiempo de escritura con coalesce: {end - start:.2f} segundos")

# Mantener la sesión abierta para observación en UI si es necesario
time.sleep(120)

# 🧪 Ejercicio Práctico - Sesión 2: Particiones, Paralelismo y Tiempos de Ejecución

## 🎯 Objetivo
Practicar el control de particiones y medir el impacto en el rendimiento a través del tiempo de escritura con diferentes configuraciones.

---

## 📝 Instrucciones

### 1️⃣ Crear un DataFrame
- Generá un `DataFrame` en PySpark con **5.000 registros** y al menos **3 columnas** (por ejemplo: número, valor aleatorio, texto).

---

### 2️⃣ Escribir en formato Parquet con 3 particiones
- Reparticionar el DataFrame en **3 particiones**.
- Medir el tiempo de escritura en formato Parquet.

---

### 3️⃣ Leer y reducir a 1 partición
- Leé los archivos `.parquet` generados.
- Usá `coalesce(1)` para reducir a **una sola partición**.

---

### 4️⃣ Escribir como CSV con compresión BZIP2
- Escribí el DataFrame como **un solo archivo CSV** comprimido en formato **bzip2**.
- Medí el tiempo de escritura.

```python
df_single.write.mode("overwrite") \
    .option("compression", "bzip2") \
    .csv("/tmp/df_csv_bzip")
```

---

## 📂 Entrega

- Subí el script `.py` o notebook `.ipynb` con el código completo.
- Incluí los tiempos de ejecución impresos y comentarios explicativos.

---

## ✅ Criterios de evaluación

- ✔ Uso correcto de `repartition` y `coalesce`.
- ✔ Escritura en los formatos y compresión solicitados.
- ✔ Medición de tiempo correcta en cada escritura.
- ✔ Código limpio, comentado y funcional.
